In [ ]:
# default_exp protoclr
# export
import copy
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import sklearn.cluster as cluster
import torch
import torch.nn as nn
import torch.nn.functional as F
import umap
from pytorch_lightning.loggers import WandbLogger
from scipy import stats
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score
from torch.autograd import Variable
from torchvision.utils import make_grid
from tqdm.auto import tqdm

import wandb
from unsupervised_meta_learning.pl_dataloaders import UnlabelledDataModule
from unsupervised_meta_learning.proto_utils import (CAE, Decoder4L, Encoder4L,
                                                    get_images_labels_from_dl,
                                                    get_prototypes,
                                                    prototypical_loss)


In [ ]:
#export
class Classifier(nn.Module):
    def __init__(self, dim, n_way):
        super(Classifier, self).__init__()

        self.fc = nn.Linear(dim, n_way)

    def forward(self, x):
        x = self.fc(x)
        return x

    def _set_params(self, weight, bias):
        state_dict = dict(weight=weight, bias=bias)
        self.fc.load_state_dict(state_dict)

    def init_params_from_prototypes(self, z_support, n_way, n_support):
        z_support   = z_support.contiguous()
        z_proto     = z_support.view(n_way, n_support, -1 ).mean(1) #the shape of z is [n_data, n_dim]
        # Interpretation of ProtoNet as linear layer (see Snell et al. (2017))
        self._set_params(weight=2*z_proto, bias=-torch.norm(z_proto, dim=-1)**2)

In [ ]:
#export
def get_train_images(ds, num):
    return torch.stack([ds[i]['data'][0] for i in range(num)], dim=0)

class WandbImageCallback(pl.Callback):
    """
    Logs the input and output images of a module.
    """
    
    def __init__(self, input_imgs, every_n_epochs=5):
        super().__init__()
        self.input_imgs = input_imgs # Images to reconstruct during training
        self.every_n_epochs = every_n_epochs # Only save those images every N epochs (otherwise tensorboard gets quite large)
          
    def on_epoch_end(self, trainer, pl_module):
        if trainer.current_epoch % self.every_n_epochs == 0:
            # Reconstruct images
            input_imgs = self.input_imgs.to(pl_module.device)
            with torch.no_grad():
                pl_module.eval()
                _, reconst_imgs = pl_module(input_imgs)
                pl_module.train()
            
            imgs = torch.stack([input_imgs, reconst_imgs], dim=1).flatten(0,1)
            grid = make_grid(imgs, nrow=2,)#  normalize=True, range=(-1,1))
            trainer.logger.experiment.log({
                "reconstructions": wandb.Image(grid, caption='Reconstructions'),
                "global_step": trainer.global_step
            })
            # trainer.logger.experiment.add_image("Reconstructions", grid, global_step=trainer.global_step)

In [ ]:
#export
class TensorBoardImageCallback(pl.Callback):
    def __init__(self, input_imgs, every_n_epochs=5):
        super().__init__()
        self.input_imgs = input_imgs # Images to reconstruct during training
        self.every_n_epochs = every_n_epochs # Only save those images every N epochs (otherwise tensorboard gets quite large)

    def on_epoch_end(self, trainer, pl_module):
        if trainer.current_epoch % self.every_n_epochs == 0:
            # Reconstruct images
            input_imgs = self.input_imgs.to(pl_module.device)
            with torch.no_grad():
                pl_module.eval()
                _, reconst_imgs = pl_module(input_imgs)
                pl_module.train()
            
            imgs = torch.stack([input_imgs, reconst_imgs], dim=1).flatten(0,1)
            grid = make_grid(imgs, nrow=2,)#  normalize=True, range=(-1,1))
            trainer.logger.experiment.add_image("Reconstructions", grid, global_step=trainer.global_step)

In [ ]:
#export
class ConfidenceIntervalCallback(pl.Callback):
    def __init__(self) -> None:
        super().__init__()
        self.losses = []
        self.accuracies = []
    def on_test_batch_end(self, trainer , pl_module , outputs, batch, batch_idx, dataloader_idx) -> None:
        loss, accuracy = outputs
        self.losses.append(loss)
        self.accuracies.append(accuracy)
    def on_test_end(self, trainer, pl_module) -> None:
        conf_interval = stats.t.interval(0.95, len(self.accuracies)-1, loc=np.mean(self.accuracies),
                                      scale=stats.sem(self.accuracies))
        
        wandb.log({'Confidence Interval': conf_interval}, step=trainer.global_step)

        plt.ylabel("Average Test Accuracy")
        plt.errorbar([1], np.mean(self.accuracies), yerr=np.std(self.accuracies), fmt='o', color='black',
             ecolor='lightgray', elinewidth=3, capsize=0)
        wandb.log({
            'Average Test Accuracy with std dev': wandb.Image(plt)
        }, step=trainer.global_step)

In [ ]:
# export
class UMAPCallback(pl.Callback):
    # currently only works with wandb
    def __init__(self, image_f, every_n_epochs=10, logger='wandb') -> None:
        super().__init__()
        self.image_f = image_f
        self.every_n_epochs = every_n_epochs
        self.logging_tech = logger
    def on_validation_end(self, trainer: 'pl.Trainer', pl_module: 'pl.LightningModule') -> None:
        if trainer.current_epoch % self.every_n_epochs == 0:
            imgs, labels = self.image_f()
            imgs, labels = imgs.to(pl_module.device), labels.to(labels.device)
            with torch.no_grad():
                pl_module.eval()
                z, _ = pl_module(imgs)
                pl_module.train()
            z = z.detach().cpu().tolist()
            xs = umap.UMAP(random_state=42).fit_transform(z)
            plt.scatter(xs[:, 0], xs[:, 1], s=0.5, c=labels, cmap='Spectral')
            if self.logging_tech == 'wandb':
                wandb.log({
                    'UMAP clustering of embeddings': wandb.Image(plt),
                }, step=trainer.global_step)
            elif self.logging_tech == 'tb':
                pass

        # return super().on_validation_end(trainer, pl_module)

In [ ]:
# export
class ProtoCLR(pl.LightningModule):
    def __init__(self, n_support, n_query, batch_size,
                 lr_decay_step, lr_decay_rate, dataset='omniglot',
                 num_input_channels=1, base_channel_size=64, latent_dim=64,
                 encoder_class=Encoder4L, decoder_class=Decoder4L,
                 classifier=None, gamma=5.0, lr=1e-3, inner_lr=1e-3,
                 ae=False, distance='euclidean', τ=.5, mode='trainval', eval_ways=5,
                 sup_finetune=True, sup_finetune_lr=1e-3, sup_finetune_epochs=15,
                 ft_freeze_backbone=True, finetune_batch_norm=False, log_images=True):
        super().__init__()

        self.encoder = encoder_class(
            num_input_channels, base_channel_size, latent_dim)
        self.decoder = decoder_class(
            num_input_channels, base_channel_size, latent_dim)

        # self.model = model
        self.ae = ae
        self.batch_size = batch_size
        self.n_support = n_support
        self.n_query = n_query

        self.distance = distance
        self.τ = τ

        # gamma will be used to weight the values of the MSE loss to potentially bring it up to par
        # gamma can also be adaptive in the future
        self.gamma = gamma
        self.lr = lr
        self.lr_decay_rate = lr_decay_rate
        self.lr_decay_step = lr_decay_step
        self.inner_lr = inner_lr

        self.mode = mode
        self.eval_ways = eval_ways
        self.sup_finetune = sup_finetune
        self.sup_finetune_lr = sup_finetune_lr
        self.sup_finetune_epochs = sup_finetune_epochs
        self.ft_freeze_backbone = ft_freeze_backbone
        self.finetune_batch_norm = finetune_batch_norm

        self.log_images = log_images

        # self.example_input_array = [batch_size, 1, 28, 28] if dataset == 'omniglot'\
        #     else [batch_size, 3, 84, 84]

        self.automatic_optimization = False

    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=.01)
        sch = torch.optim.lr_scheduler.StepLR(
            opt, step_size=self.lr_decay_step, gamma=self.lr_decay_rate)
        return {'optimizer': opt, 'lr_scheduler': sch}

    def forward(self, x):
        z = self.encoder(x.view(-1, *x.shape[-3:]))
        embeddings = nn.Flatten()(z)
        recons = self.decoder(z)
        return embeddings.view(*x.shape[:-3], -1), recons.view(*x.shape)


    def _get_pixelwise_reconstruction_loss(self, x, x_hat, ways):
        mse_loss = F.mse_loss(x.squeeze(0), x_hat.squeeze(
            0), reduction='none').sum(dim=[1, 2, 3, ]).mean(dim=[0])
        return mse_loss

    def calculate_protoclr_loss(self, z, y_support, y_query, ways):

        #
        # e.g. [1,50*n_support,*(3,84,84)]
        z_support = z[:, :ways * self.n_support]
        # e.g. [1,50*n_query,*(3,84,84)]
        z_query = z[:, ways * self.n_support:]
        # Get prototypes
        if self.n_support == 1:
            z_proto = z_support  # in 1-shot the prototypes are the support samples
        else:
            z_proto = get_prototypes(z_support, y_support, ways)

        loss, accuracy = prototypical_loss(z_proto, z_query, y_query,
                                           distance=self.distance, τ=self.τ)
        return loss, accuracy

    def training_step(self, batch, batch_idx):
        opt = self.optimizers()
        sch = self.lr_schedulers()

        # [batch_size x ways x shots x image_dim]
        data = batch['data'].to(self.device)
        data = data.unsqueeze(0)
        # e.g. 50 images, 2 support, 2 query, miniImageNet: torch.Size([1, 50, 4, 3, 84, 84])
        batch_size = data.size(0)
        ways = data.size(1)

        # Divide into support and query shots
        x_support = data[:, :, :self.n_support]
        # e.g. [1,50*n_support,*(3,84,84)]
        x_support = x_support.reshape(
            (batch_size, ways * self.n_support, *x_support.shape[-3:]))
        x_query = data[:, :, self.n_support:]
        # e.g. [1,50*n_query,*(3,84,84)]
        x_query = x_query.reshape(
            (batch_size, ways * self.n_query, *x_query.shape[-3:]))

        # Create dummy query labels
        y_query = torch.arange(ways).unsqueeze(
            0).unsqueeze(2)  # batch and shot dim
        y_query = y_query.repeat(batch_size, 1, self.n_query)
        y_query = y_query.view(batch_size, -1).to(self.device)

        y_support = torch.arange(ways).unsqueeze(
            0).unsqueeze(2)  # batch and shot dim
        y_support = y_support.repeat(batch_size, 1, self.n_support)
        y_support = y_support.view(batch_size, -1).to(self.device)

        # Extract features (first dim is batch dim)
        # e.g. [1,50*(n_support+n_query),*(3,84,84)]
        x = torch.cat([x_support, x_query], 1)
        z, x_hat = self.forward(x)

        opt.zero_grad()

        loss, accuracy = self.calculate_protoclr_loss(
            z, y_support, y_query, ways)

        self.log('clr_loss', loss.item(), prog_bar=True)
        # adding the pixelwise reconstruction loss at the end
        # it has been broadcasted such that each support source image is broadcasted thrice over the three
        # query set images - which are the augmentations of the support image
        if self.ae:
            mse_loss = self._get_pixelwise_reconstruction_loss(
                x, x_hat, ways) * self.gamma
            self.log('mse_loss', mse_loss.item(), prog_bar=True,)
            loss += mse_loss

        self.manual_backward(loss)
        opt.step()
        sch.step()

        self.log_dict({
            'loss': loss.item(),
            'train_accuracy': accuracy
        }, prog_bar=True)

        return loss, accuracy

    @torch.enable_grad()
    def supervised_finetuning(self, encoder, episode, device='cpu', proto_init=True,
                              freeze_backbone=False, finetune_batch_norm=False,
                              inner_lr=0.001, total_epoch=15, n_way=5):
        x_support = episode['train'][0][0]  # only take data & only first batch
        x_support = x_support.to(device)
        x_support_var = Variable(x_support)
        x_query = episode['test'][0][0]  # only take data & only first batch
        x_query = x_query.to(device)
        x_query_var = Variable(x_query)
        n_support = x_support.shape[0] // n_way
        n_query = x_query.shape[0] // n_way

        batch_size = n_way
        support_size = n_way * n_support

        y_a_i = Variable(torch.from_numpy(np.repeat(range(n_way), n_support))).to(
            self.device)  # (25,)

        x_b_i = x_query_var
        x_a_i = x_support_var
        encoder.eval()

        # TODO: remove view add in network
        z_a_i = nn.Flatten()(encoder(x_a_i.to(device)))  # .view(*x_a_i.shape[:-3], -1)
        encoder.train()

        # Define linear classifier
        input_dim = z_a_i.shape[1]
        classifier = Classifier(input_dim, n_way=n_way)
        classifier.to(device)
        classifier.train()
        ###############################################################################################
        loss_fn = nn.CrossEntropyLoss().to(device)
        # Initialise as distance classifer (distance to prototypes)
        if proto_init:
            classifier.init_params_from_prototypes(z_a_i, n_way, n_support)
        classifier_opt = torch.optim.Adam(classifier.parameters(), lr=inner_lr)
        if freeze_backbone is False:
            delta_opt = torch.optim.Adam(
                filter(lambda p: p.requires_grad, encoder.parameters()), lr=inner_lr)
        # Finetuning
        if freeze_backbone is False:
            encoder.train()
        else:
            encoder.eval()
        classifier.train()
        if not finetune_batch_norm:
            for module in encoder.modules():
                if isinstance(module, torch.nn.modules.BatchNorm2d):
                    module.eval()

        for epoch in tqdm(range(total_epoch), total=total_epoch, leave=False):
            rand_id = np.random.permutation(support_size)

            for j in range(0, support_size, batch_size):
                classifier_opt.zero_grad()
                if freeze_backbone is False:
                    delta_opt.zero_grad()

                #####################################
                selected_id = torch.from_numpy(
                    rand_id[j: min(j+batch_size, support_size)]).to(device)

                z_batch = x_a_i[selected_id]
                y_batch = y_a_i[selected_id]
                #####################################

                output = nn.Flatten()(encoder(z_batch))
                output = classifier(output)
                loss = loss_fn(output, y_batch)

                #####################################
                loss.backward()

                classifier_opt.step()

                if freeze_backbone is False:
                    delta_opt.step()
        classifier.eval()
        encoder.eval()

        output = nn.Flatten()(encoder(x_b_i.to(device)))
        scores = classifier(output)

        y_query = torch.tensor(np.repeat(range(n_way), n_query)).to(device)
        loss = F.cross_entropy(scores, y_query, reduction='mean')
        _, predictions = torch.max(scores, dim=1)
        accuracy = torch.mean(predictions.eq(y_query).float())
        return loss, accuracy.item()

    def validation_step(self, batch, batch_idx):
        original_encoder_state = copy.deepcopy(self.encoder.state_dict())
        if not self.mode == 'trainval':
            original_encoder_state = copy.deepcopy(self.encoder.state_dict())

        if self.sup_finetune:
            loss, accuracy = self.supervised_finetuning(self.encoder,
                                                        episode=batch,
                                                        inner_lr=self.sup_finetune_lr,
                                                        total_epoch=self.sup_finetune_epochs,
                                                        freeze_backbone=self.ft_freeze_backbone,
                                                        finetune_batch_norm=self.finetune_batch_norm,
                                                        device=self.device,
                                                        n_way=self.eval_ways)
            self.encoder.load_state_dict(original_encoder_state)
        elif self.mode == 'trainval':
            with torch.no_grad():
                loss, accuracy, _, _ = self.calculate_protoclr_loss(
                    batch, ae=False)
        else:
            with torch.no_grad():
                loss, accuracy, _, _ = self.calculate_protoclr_loss(
                    batch, ae=False)
        self.log_dict({
            'val_loss': loss.detach(),
            'val_accuracy': accuracy
        }, prog_bar=True)

        return loss.item(), accuracy

    def test_step(self, batch, batch_idx):
        original_encoder_state = copy.deepcopy(self.encoder.state_dict())
        # if self.sup_finetune:
        loss, accuracy = self.supervised_finetuning(self.encoder,
                                                    episode=batch,
                                                    inner_lr=self.sup_finetune_lr,
                                                    total_epoch=self.sup_finetune_epochs,
                                                    freeze_backbone=self.ft_freeze_backbone,
                                                    finetune_batch_norm=self.finetune_batch_norm,
                                                    device=self.device,
                                                    n_way=self.eval_ways)
        self.encoder.load_state_dict(original_encoder_state)
        self.log("test_loss", loss.item(), on_step=True,
                 on_epoch=True, prog_bar=True, logger=True)
        self.log("test_acc", accuracy, on_step=True,
                 on_epoch=True, prog_bar=True, logger=True)
        return loss.item(), accuracy


In [ ]:
dm = UnlabelledDataModule('omniglot', './data/untarred/', split='train', transform=None,
                 n_support=1, n_query=3, n_images=None, n_classes=None, batch_size=50,
                 seed=10, mode='trainval', merge_train_val=True)

In [ ]:
model = ProtoCLR(model=CAE(1, 64, hidden_size=64), n_support=1, n_query=3, batch_size=50, lr_decay_step=25000, lr_decay_rate=.5, ae=True)

In [ ]:
logger = WandbLogger(
    project='ProtoCLR+AE',
    config={
        'batch_size': 50,
        'steps': 100,
        'dataset': "omniglot"
    }
)
trainer = pl.Trainer(
        profiler='simple',
        max_epochs=10,
        limit_train_batches=100,
        fast_dev_run=False,
        num_sanity_val_steps=2, gpus=1, #logger=logger
    )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    trainer.fit(model, datamodule=dm)

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 01_nn_utils.ipynb.
Converted 01b_data_loaders_pl.ipynb.
Converted 01c_grad_utils.ipynb.
Converted 01d_proto_utils.ipynb.
Converted 02_maml_pl.ipynb.
Converted 02b_iMAML.ipynb.
Converted 03_protonet_pl.ipynb.
Converted 03b_ProtoCLR.ipynb.
Converted 04_cactus.ipynb.
Converted jax-maml.ipynb.
Converted lightning-vae.ipynb.
Converted unsupervised_loaders.ipynb.
Converted index.ipynb.
Converted scratch.ipynb.
